# Convert a video from hdf5 to a colored max projection

In [2]:
import h5py
import numpy as np
import cv2
import matplotlib.pyplot as plt
import imageio

In [3]:
def colorize_frame(frame, cmap, z_val, threshold_val=None):
    """
    Colors a frame according a colormap, separated by a threshold
    
    This will REMOVE information related to intensity (for now), and replace everything with cmap(z_val)
    """
    
    if threshold_val is None:
        threshold_val = np.median(frame)

    sz = frame.shape
    new_frame = np.zeros(sz[0:2] + (4,), dtype='uint8')
    mask = frame > threshold_val
    #mask_ind = np.where(mask)
    new_val = (np.array(cmap(z_val))*255).astype('uint8') # Everything else is zeros
    new_frame[mask] = new_val
    
    return new_frame, threshold_val

####
####
####
def colorize_stack(stack, cmap, threshold_val=None):
    """
    Colors a stack by taking a maximum projection, and then colorizing each entry according to the original z-stack
    If the intensity is less than a threshold, it will be set to 0
    
    This will REMOVE information related to intensity (for now), and replace everything with cmap(z_val)
    """
    #if threshold_val is None:
     #   threshold_val = np.max(stack[0,...])/1.1

    sz = stack.shape
    #print(sz)
    new_frame = np.zeros(sz[0:2] + (4,), dtype='uint8')
    #print(new_frame.shape)
    
    # First, do a max projection
    max_proj = np.max(stack, axis=-1)
    max_proj_ind = np.argmax(stack, axis=-1)
    #print(max_proj)
    #print(max_proj_ind)
    #print(max_proj_ind.shape)
    if threshold_val is None:
        threshold_val = 0.1*np.max(max_proj)
    
    # Colorize the projection
    for i_z in range(sz[-1]):
        mask_val = max_proj > threshold_val
        mask_z = i_z == max_proj_ind
        mask = mask_val & mask_z #np.where(mask_val & mask_z)
        #print(mask.shape)
        new_val = (np.array(cmap(i_z/sz[-1]))*255).astype('uint8') # Everything else is zeros
        new_frame[mask] = new_val
    
    #new_frame[:,:,-1] = max_proj.astype('uint8') # Replace alpha with intensity information
    
    return new_frame, threshold_val


####
####
####
def colorize_stack_by_channels(stack, threshold_val=None):
    """
    Colors a stack by taking a maximum projection, and then colorizing the RGB channels according thirds of the stack
    i.e. Red channel = max(stack[:,:,0:10]) etc.
    
    This will REMOVE information related to intensity (for now), and replace everything with cmap(z_val)
    """
    if threshold_val is None:
        threshold_val = np.max(stack[0,...])/1.1

    sz = stack.shape
    new_frame = 255*np.ones(sz[0:2] + (4,), dtype='uint8')
    #print(new_frame.shape)
    #print(stack.shape)
    
    # First, do 3 max projections
    max_proj0 = np.max(stack[...,0:10], axis=-1)
    #max_proj_ind1 = np.argmax(stack[0:10], axis=-1)
    max_proj1 = np.max(stack[...,11:20], axis=-1)
    #max_proj_ind2 = np.argmax(stack[11:20], axis=-1)
    max_proj2 = np.max(stack[...,21:30], axis=-1)
    #max_proj_ind3 = np.argmax(stack[21:30], axis=-1)
    #print(max_proj0.shape)
    
    # Colorize each channel
    new_frame[:,:,0] = max_proj0.astype('uint8')
    new_frame[:,:,1] = max_proj1.astype('uint8')
    new_frame[:,:,2] = max_proj2.astype('uint8')
    
    return new_frame, threshold_val


In [6]:
# Take a subset of data
num_frames = 200

# File location
in_fname = '/groups/zimmer/Ulises/wbfm/wbfm_december/20181220/data/worm1/hdf5/'
which_file = 'mCherry' # Is also the name of the field in the file
in_fname = in_fname + which_file + '.hdf5'

# Colormap for z slices
cmap = plt.get_cmap('plasma')
#cmap_background = plt.get_cmap('gray')

# Output location and opencv writer
out_fname = which_file + '-colored_max_projection-test.mp4'
#fourcc = cv2.VideoWriter_fourcc(*'MJPG')
fourcc = cv2.VideoWriter_fourcc(*'H264')
#fourcc = cv2.VideoWriter_fourcc(*'IYUV')
#fourcc = cv2.VideoWriter_fourcc(*'XVID')
#fourcc = cv2.VideoWriter_fourcc(*'DIVX')
#writer = cv2.VideoWriter(out_fname, fourcc, 5, (650,850))
#writer = cv2.VideoWriter(out_fname, -1, 5, (650,850))

with h5py.File(in_fname, 'r') as x_load:
    volumes = np.squeeze(x_load[which_file][0:num_frames, :, :, :, :])
    volumes = np.moveaxis(volumes, 1,3) # Last axis is the colormap
    
    out_tmp = []
    
    # Only want the actual neurons (approximately) to be colored; everything else should stay black
    for i_frame in range(volumes.shape[0]):
        #tmp, thresh = colorize_stack(volumes[i_frame,:], cmap)
        tmp, thresh = colorize_stack_by_channels(volumes[i_frame,:])
        #for i_z in range(stack.shape[-1]):
         #   this_frame = stack[i_frame,:,:,i_z]
         #   tmp, _ = colorize(this_frame, cmap, i_z)
        #print(tmp.shape)
        #print(thresh)
        #print(tmp)
        #plt.imshow(tmp, cmap='gray')
        #plt.show()
        #writer.write(tmp)
        out_tmp.append(tmp)
    
    #print(frame.shape)

    #for frame in range(num_frames):
    #    writer.write(data).astype('uint8'))

#writer.release()

In [7]:
#print(out_tmp.shape)
imageio.mimwrite(out_fname, out_tmp, fps=5)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (850, 650) to (864, 656) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


In [ ]:
# Check to make sure it worked
try:
    cap.release()
    cap = cv2.VideoCapture(out_fname) 
except:
    cap = cv2.VideoCapture(out_fname)

    cv2.destroyAllWindows()

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    print(frame)

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Display the resulting frame
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [ ]:
with h5py.File(in_fname, 'r') as x_load:
    stack = np.squeeze(x_load[which_file][0:num_frames, :, :, :, :])
    stack = np.moveaxis(stack, 1,3) # Last axis is the colormap
    
    dat = stack[1,...]
    print(dat.shape)
    sz = dat.shape
    print((sz[0:2]+ (3,)))
    tmp = np.zeros(sz[0:2]+ (3,))
    print(tmp.shape)

print(cmap(0.5))